In [1]:
import json, sys, os, requests
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

In [3]:
def get_ons_data_ts(dataset, series, save=True, api_path="https://api.ons.gov.uk"):
    r = requests.get(
        api_path
        + "/timeseries/"
        + series.lower()
        + "/dataset/"
        + dataset.lower()
        + "/data"
    )
    content = json.loads(r.content)
    if save:
        print("Saving DATASET", dataset, "SERIES", series, "to local mirror...")
        open("../../datasets/ons/" + dataset + "/" + series + ".json", "w").write(
            json.dumps(content)
        )
        print("OK!")
    return content

Employment

In [23]:
dataset = "lms"
data = {}
series = ["mgsx", "lf24", "kac3", "lf2s"]
series_colors = {
    "mgsx": "eco-pink",
    "lf24": "eco-turquiose",
    "kac3": "eco-yellow",
    "lf2s": "eco-mid-blue",
}
series_offset = {"mgsx": 0.8, "lf24": 0.5, "kac3": 1, "lf2s": 0.9}
series_domain = {"mgsx": [2, 12], "lf24": [65, 80], "kac3": [-2, 6], "lf2s": [20, 26]}
series_domain2 = {
    "mgsx": [3.5, 5.5],
    "lf24": [74, 77],
    "kac3": [-2, 6],
    "lf2s": [20, 21.5],
}
series_titles = {
    "mgsx": "Unemployment",
    "lf24": "Employment",
    "kac3": "Wage growth",
    "lf2s": "Economic inactivity rate",
}
series_subtitles = {
    "mgsx": "Rate (%), age 16+ seasonally adjusted",
    "lf24": "Rate (%), age 16+ seasonally adjusted",
    "kac3": "Weekly earnings, three month average, y-o-y growth (%)",
    "lf2s": "UK, Aged 16-64 (%)",
}
series_text3 = {
    "mgsx": "year(datum.date)>=2021",
    "lf24": "year(datum.date)>=2021",
    "kac3": "year(datum.date)>=2021&month(datum.date)>=1",
    "lf2s": "year(datum.date)>=2021&month(datum.date)>=1",
}
for serie in series:
    d = get_ons_data_ts(dataset, serie)
    data[serie] = {
        "title": d["description"]["title"].split(")")[0] + ")",
        "data": d["months"],
    }

Saving DATASET lms SERIES mgsx to local mirror...
OK!
Saving DATASET lms SERIES lf24 to local mirror...
OK!
Saving DATASET lms SERIES kac3 to local mirror...
OK!
Saving DATASET lms SERIES lf2s to local mirror...
OK!


Now push datasets to GitHub.

In [26]:
for l in ["live", "eco", "local"]:
    layers = []
    for serie in series:
        if l == "live":
            url = (
                "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
                + serie
                + "/dataset/"
                + dataset
                + "/data"
            )
        else:
            url = (
                "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
                + dataset
                + "/"
                + serie
                + ".json"
            )
        if l == "local":
            data = alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        else:
            data = pd.DataFrame(json.loads(requests.get(url).content)["months"])
            data["date"] = pd.to_datetime(data["date"])
        print("Processing", serie, "for", l, "...")
        base = (
            alt.Chart(data)
            .transform_calculate(serie=repr(serie))
            .transform_calculate(label="datum.year+' '+datum.month")
            .transform_calculate(date="toDate(datum.year+' '+datum.month+' '+1)")
            .encode(x=alt.X("date:T", axis=alt.Axis(grid=True, title="")))
        )
        area = base.mark_area(
            interpolate="monotone",
            fillOpacity=0.7,
            stroke=colors["eco-gray"],
            strokeWidth=0.5,
            color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors[series_colors[serie]], offset=series_offset[serie]
                    ),
                ],
                x1=0.8,
                x2=1,
                y1=1,
                y2=0,
            ),
        ).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        )
        line = base.mark_line(color=colors[series_colors[serie]]).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                scale=alt.Scale(domain=series_domain2[serie]),
            )
        )
        # Create a selection that chooses the nearest point & selects based on x-value
        nearest = alt.selection(
            type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
        )
        # Transparent selectors across the chart. This is what tells us
        # the x-value of the cursor
        selectors = (
            base.mark_point()
            .encode(
                opacity=alt.value(0),
            )
            .add_selection(nearest)
        )

        # Draw points on the line, and highlight based on selection
        points = area.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )

        # Draw text labels near the points, and highlight based on selection
        text = area.mark_text(align="left", dx=-25, dy=35).encode(
            text=alt.condition(nearest, "value:N", alt.value(" "))
        )

        # Draw text labels near the points, and highlight based on selection
        text2 = area.mark_text(align="right", dx=5, dy=10,).encode(
            text=alt.condition(nearest, "label:N", alt.value(" ")),
            x=alt.value(390),
            y=alt.value(0),
        )

        # Draw text labels near the points, and highlight based on selection
        text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
            text=alt.condition(series_text3[serie], "value:Q", alt.value(" ")),
            x=alt.value(92),
            y=alt.value(3),
        )

        # Draw a rule at the location of the selection
        rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

        # Put the five layers into a chart and bind the data
        layer1 = alt.layer(area, selectors, points, rules, text, text2,).properties(
            width=400,
            height=150,
            title={"text": series_titles[serie], "subtitle": series_subtitles[serie]},
        )
        layer2 = (
            alt.layer(
                line,
                selectors,
                points,
                rules,
                text,
                text3,
            )
            .properties(width=100, height=150)
            .transform_filter("datum.date>='2020'")
        )
        layers.append(alt.hconcat(layer1, layer2, spacing=20))

    vlayers = alt.vconcat(
        layers[0], layers[1], layers[2], layers[3]
    )  # .configure_view(stroke=None)
    vlayers.save(dataset + "_" + l + ".json")
    print("OK!")

Processing mgsx for live ...
Processing lf24 for live ...
Processing kac3 for live ...
Processing lf2s for live ...
OK!
Processing mgsx for eco ...
Processing lf24 for eco ...
Processing kac3 for eco ...
Processing lf2s for eco ...
OK!
Processing mgsx for local ...
Processing lf24 for local ...
Processing kac3 for local ...
Processing lf2s for local ...
OK!


In [27]:
vlayers

alt.VConcatChart(...)

In [28]:
dataset = "lms"
data = {}
series = ["lf2s", "ybtm", "lf2t"]
series_colors = {"lf2s": "eco-pink", "ybtm": "eco-turquiose", "lf2t": "eco-yellow"}
series_offset = {"lf2s": 0.8, "ybtm": 0.5, "lf2t": 1}
series_domain = {"lf2s": [2, 12], "ybtm": [65, 80], "lf2t": [-2, 6]}
series_domain2 = {"lf2s": [20, 22], "ybtm": [15, 18], "lf2t": [24, 25]}
series_titles = {
    "lf2s": "Inactivity Rate",
    "ybtm": "Inactivity Rate",
    "lf2t": "Inactivity Rate",
}
series_subtitles = {
    "lf2s": "(%), All",
    "ybtm": "(%), Men",
    "lf2t": "(%), Women",
}
series_text3 = {
    "lf2s": "year(datum.date)>=2021",
    "ybtm": "year(datum.date)>=2021",
    "lf2t": "year(datum.date)>=2021&month(datum.date)>=1",
}
for serie in series:
    d = get_ons_data_ts(dataset, serie)
    data[serie] = {
        "title": d["description"]["title"].split(")")[0] + ")",
        "data": d["months"],
    }

Saving DATASET lms SERIES lf2s to local mirror...
OK!
Saving DATASET lms SERIES ybtm to local mirror...
OK!
Saving DATASET lms SERIES lf2t to local mirror...
OK!


Economic inactivity rate

In [29]:
for l in ["live", "eco", "local"]:
    layers = []
    for serie in series:
        if l == "live":
            url = (
                "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
                + serie
                + "/dataset/"
                + dataset
                + "/data"
            )
        else:
            url = (
                "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
                + dataset
                + "/"
                + serie
                + ".json"
            )
        if l == "local":
            data = alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        else:
            data = pd.DataFrame(json.loads(requests.get(url).content)["months"])
            data["date"] = pd.to_datetime(data["date"])
        print("Processing", serie, "for", l, "...")
        base = (
            alt.Chart(data)
            .transform_calculate(serie=repr(serie))
            .transform_calculate(label="datum.year+' '+datum.month")
            .transform_calculate(date="toDate(datum.year+' '+datum.month+' '+1)")
            .encode(x=alt.X("date:T", axis=alt.Axis(grid=True, title="")))
        )
        area = base.mark_area(
            interpolate="monotone",
            fillOpacity=0.7,
            stroke=colors["eco-gray"],
            strokeWidth=0.5,
            color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors[series_colors[serie]], offset=series_offset[serie]
                    ),
                ],
                x1=0.8,
                x2=1,
                y1=1,
                y2=0,
            ),
        ).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        )
        line = base.mark_line(color=colors[series_colors[serie]]).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                scale=alt.Scale(domain=series_domain2[serie]),
            )
        )
        # Create a selection that chooses the nearest point & selects based on x-value
        nearest = alt.selection(
            type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
        )
        # Transparent selectors across the chart. This is what tells us
        # the x-value of the cursor
        selectors = (
            base.mark_point()
            .encode(
                opacity=alt.value(0),
            )
            .add_selection(nearest)
        )

        # Draw points on the line, and highlight based on selection
        points = area.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )

        # Draw text labels near the points, and highlight based on selection
        text = area.mark_text(align="left", dx=-25, dy=35).encode(
            text=alt.condition(nearest, "value:N", alt.value(" "))
        )

        # Draw text labels near the points, and highlight based on selection
        text2 = area.mark_text(align="right", dx=5, dy=10,).encode(
            text=alt.condition(nearest, "label:N", alt.value(" ")),
            x=alt.value(390),
            y=alt.value(0),
        )

        # Draw text labels near the points, and highlight based on selection
        text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
            text=alt.condition(series_text3[serie], "value:Q", alt.value(" ")),
            x=alt.value(92),
            y=alt.value(3),
        )

        # Draw a rule at the location of the selection
        rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

        # Put the five layers into a chart and bind the data
        layer1 = alt.layer(area, selectors, points, rules, text, text2,).properties(
            width=400,
            height=150,
            title={"text": series_titles[serie], "subtitle": series_subtitles[serie]},
        )
        layer2 = (
            alt.layer(
                line,
                selectors,
                points,
                rules,
                text,
                text3,
            )
            .properties(width=100, height=150)
            .transform_filter("datum.date>='2020'")
        )
        layers.append(alt.hconcat(layer1, layer2, spacing=20))

    layers = alt.vconcat(
        layers[0], layers[1], layers[2]
    )  # .configure_view(stroke=None)
    layers.save(dataset + "2_" + l + ".json")
    print("OK!")
layers

Processing lf2s for live ...
Processing ybtm for live ...
Processing lf2t for live ...
OK!
Processing lf2s for eco ...
Processing ybtm for eco ...
Processing lf2t for eco ...
OK!
Processing lf2s for local ...
Processing ybtm for local ...
Processing lf2t for local ...
OK!


alt.VConcatChart(...)

In [30]:
ages = {
    "mgvf": "16-24",
    "ycgm": "25-34",
    "ycgs": "35-49",
    "lf28": "50-64",
    "k5hu": "65+",
}
dfs = []
for s in ages:
    print(s)
    df = pd.DataFrame(get_ons_data_ts("lms", s)["months"])
    df["series"] = s
    df["age"] = ages[s]
    df["date"] = pd.to_datetime(df["date"])
    dfs.append(df)

mgvf
Saving DATASET lms SERIES mgvf to local mirror...
OK!
ycgm
Saving DATASET lms SERIES ycgm to local mirror...
OK!
ycgs
Saving DATASET lms SERIES ycgs to local mirror...
OK!
lf28
Saving DATASET lms SERIES lf28 to local mirror...
OK!
k5hu
Saving DATASET lms SERIES k5hu to local mirror...
OK!


In [31]:
df = pd.concat(dfs)

In [32]:
df = (
    df.set_index("date")
    .sort_index()
    .loc["2019-10-01":]
    .reset_index()
    .set_index(["age", "date"])["value"]
    .unstack()
    .astype(int)
)

In [33]:
for c in df.columns[1:]:
    df[c] = df[df.columns[0]] - df[c]
df[df.columns[0]] = 0

In [34]:
df = df.stack().reset_index()
df.columns = ["age", "date", "value"]

In [35]:
f = "unemployment_by_age"
df.to_csv("../../datasets/eco/unemployment/" + f + ".csv")

In [36]:
color_scale = [
    colors["eco-green"],
    colors["eco-turquiose"],
    colors["eco-light-blue"],
    colors["eco-mid-blue"],
    colors["eco-blue"],
]

In [37]:
base = alt.Chart(df).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="thousand",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=295,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=6,
            format=".0f",
        ),
    ),
    x=alt.X(
        "date:T",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            labelExpr="datum.value<toDate('2019-11-01')?'':month(datum.value)==0?timeFormat(datum.value,'%Y'):timeFormat(datum.value,'%B')",
            labelAngle=0,
        ),
    ),
    color=alt.Color("age:N", legend=None, scale=alt.Scale(range=color_scale)),
)
lines = base.mark_line()
labels = (
    base.mark_text(align="left", dx=5)
    .encode(text=alt.Text("age:N"))
    .transform_filter("datum.date>toDate('2020-12-01')")
)
layer = lines + labels
layer.configure_view(stroke=None).properties(title="").properties(height=300, width=400)
layer.save("../../panels/unemployment/" + f + ".json")
layer

alt.LayerChart(...)

In [46]:
pop = (
    pd.read_excel("../../datasets/ons/pop/population_pyramid.xlsx", skiprows=1)
    .set_index("variable")
    .loc[["UNITED KINGDOM"]]
)

In [48]:
np.sum([pop["m_01_16"], pop["m_01_17"]])

767574.0

In [60]:
pop["16-24"] = np.sum(
    [pop[m + "_01_" + str(i)] for m in ["f", "m"] for i in range(16, 25)]
)
pop["35-49"] = np.sum(
    [pop[m + "_01_" + str(i)] for m in ["f", "m"] for i in range(35, 50)]
)
pop["50-64"] = np.sum(
    [pop[m + "_01_" + str(i)] for m in ["f", "m"] for i in range(50, 65)]
)
pop["25-34"] = np.sum(
    [pop[m + "_01_" + str(i)] for m in ["f", "m"] for i in range(25, 35)]
)
pop["65+"] = np.sum(
    [pop[m + "_01_" + str(i)] for m in ["f", "m"] for i in range(65, 91)]
)

In [82]:
df2 = df.join(
    pd.DataFrame(
        pop[["16-24", "35-49", "50-64", "25-34", "65+"]].stack().loc["UNITED KINGDOM"],
        columns=["pop"],
    ),
    on="age",
)
df2["vp"] = df2["value"]*1000 / df2["pop"]

In [83]:
f = "unemployment_by_age_norm_by_pop"
df2.to_csv("../../datasets/eco/unemployment/" + f + ".csv")

In [85]:
base = alt.Chart(df2).encode(
    y=alt.Y(
        "vp:Q",
        axis=alt.Axis(
            grid=True,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=295,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=6,
            format=".2%",
        ),
    ),
    x=alt.X(
        "date:T",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            labelExpr="datum.value<toDate('2019-11-01')?'':month(datum.value)==0?timeFormat(datum.value,'%Y'):timeFormat(datum.value,'%B')",
            labelAngle=0,
        ),
    ),
    color=alt.Color("age:N", legend=None, scale=alt.Scale(range=color_scale)),
)
lines = base.mark_line()
labels = (
    base.mark_text(align="left", dx=5)
    .encode(text=alt.Text("age:N"))
    .transform_filter("datum.date>toDate('2020-12-01')")
)
layer = lines + labels
layer.configure_view(stroke=None).properties(title="").properties(height=300, width=400)
layer.save("../../panels/unemployment/" + f + ".json")
layer

alt.LayerChart(...)